# 03. Retention
- 여기서 말하는 리텐션은 코호트 리텐션을 말합니다
- 리텐션을 무식하게 뽑을 땐 with 구문을 사용해 join하고 count(case when first_date = event_date-1) as day1 이런식으로 무식하게 Count했습니다!
- 하지만 여기선 세련되게 함수를 사용해 진행해보겠습니다!!
- 우선 데이터셋을 아래와 같이 뽑습니다


| user_id | event_date | country | etc|
|:-------:|:----------:|:-------:|---|
|    A    | 2017-01-01 |    kr   |   |  
|    B    | 2017-01-03 |    kr   |   |  
|    A    | 2017-01-02 |    kr   |   | 

- min(event_date) : first_date로 지정
- 그리고 이 테이블을 join합니다 (self-join, 혹은 with문으로)

#### 여기서부터 Tableau 구간
#### 1. acquisition을 구합니다
~~~
DATEDIFF('day', [first_date], [event_date])
~~~
- day를 week으로 바꿀수도 있습니다! 기간을 어떻게 잡느냐에 따라 여기를 수정해주시면 됩니다

#### 2. day of retention column 생성
- 이건 표에서 컬럼으로 사용할 예정입니다
~~~
if [acquisition] = 0 then "Amount"
ELSE
"Day" + str([acquisition])
end
~~~
- acquisition이 0이면 Amount라고 명명하고, 아니라면 "Day" + str(acquisition)로 명명합니다


#### 3. retained 함수 작성
~~~
if sum([acquisition]) = 0 then
countd([user_id])
else
    -round(1-(( LOOKUP(countd([user_id]), first()) - countd([user_id]) ) / LOOKUP(countd([user_id]), FIRST())),4)
end
~~~
- sum([acquisition]) = 0이라는 것은 당일일 경우엔 countd([user_id])! 해당 first_date에 접속한 유저수를 구합니다
- 그게 아니라면 %를 구합니다!
- LOOKUP 함수의 정의는 아래와 같습니다
>‘LOOKUP(식, [오프셋])’은 지정된 상대적 오프셋만큼 현재 행으로부터 떨어진 대상 행에서 주어진 식을 계산하여 그 값을 반환한다. 대상 행을 결정할 수 없으면, ‘NULL'을 반환한다. 
또한, 'FIRST( ) + n’ 혹은 'LAST( ) - n’을 이용해서 대상을 파티션의 첫 번째 혹은 마지막 행에서부터의 상대 위치로 지정을 할 수 있다.
- [잘 쓰여진 브런치 글](https://brunch.co.kr/@cheonmyung/38)을 확인하면 한번에 이해될 거에요-! 
- LOOKUP(countd([user_id]), first())는 첫 행(first())에서 countd([user_id])를 하라는 뜻입니다! 
- round 함수는 소수점 4번째자리까지 나타내기 위해 사용했습니다
- '-'를 사용한 이유는 '서식'을 적용하기 위함입니다

#### 4. 열에 day of retention을 넣고, 행에 first_date를 넣어주세요
- 이 때 first_date는 날짜 속성이며 불연속형으로 설정해줘야 표처럼 나타납니다

#### 5. 서식 적용
- value에서 우측 마우스를 누른 후, 서식을 눌러줍니다
- 그 후 좌측 최상단에 '필드'를 누르시고 집계(retained)를 눌러주세요
- 숫자를 누르시고 사용자 지정에 #,##;0.00%;를 입력해주세요! 이것은 양수라면 1,000 이런식 음수라면 %를 사용하라는 뜻입니다!

#### 6. 필터 적용
- 기간을 유동적으로 조정하고 싶을 경우, 필터를 사용하면 편리합니다
- 단, default 설정이 따로 존재하고 유저들이 기간을 설정할 수 있도록 만들어 보겠습니다

##### 6-1) 
- 매개변수 생성 - Custom Start Date / Custome End Date를 만들어주세요 ( 데이터 유형은 날짜, 허용 가능한 값은 모두에 체크 )


##### 6-2) 
- 그 후, Date Selection Parameter라는 파라미터를 만들어줍니다
    - 데이터 유형 : 정수
    - 값에 3을 넣어주고 다음으로 표시에 "Last 2 weeks" 입력
    - 값에 4를 넣어주고 다음으로 표시에 "Custom Dates" 입력


##### 6-3) 
- 그 후 Date Selection Part라는 집계 함수를 만들어줍니다

~~~
if [매개 변수].[Date Selection Parameter] = 3 then
(if  [first_date]>= TODAY()-15 then 1 end)

elseif [매개 변수].[Date Selection Parameter] = 4 then 
(if [first_date] >= [Custom Start Date] and      
    [first_date] <= [Custom End Date] then 1 end)
END
~~~
    - Date Selection Parameter가 3이면 first_date>=TODAY()-15인 값들을 가지고 옵니다! 15인 이유는 최근 2주를 표현하는데, 제가 사용하는 빅쿼리 특성상 하루를 더 제외해야( 당일의 데이터는 내일 추출이 되기 때문에 )해서 저렇게 표시했습니다
    - Date Selection Parameter가 4면 Start Date ~ End Date 사이로 필터링합니다


##### 6-4) 
- Tableau 상단바의 '분석' - '파라미터' - Custom Start Date, Custom End Date, Date Selection Part 모두 체크해줍니다
- 기호에 맞게 설정해주세요. Date Selection Parameter는 2버튼식으로 구성하면 좋습니다

